<a href="https://colab.research.google.com/github/jeawonlll/KOHI2022_tutorials/blob/main/20220827/GBM_hands_on.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
import pandas as pd
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV, cross_val_score
from sklearn import metrics
from datetime import datetime

# Data Load
1. Mount google drive
2. Write your data path. ('/content/gdrive/My Drive/............')
3. Explore the data

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
# Write your data path
data_path = '/content/gdrive/MyDrive/Colab Notebooks/mimic_data_sample.csv'

data = pd.read_csv(data_path)

In [4]:
# Check the data size
print(data.shape)

# Check column names
print(data.columns)

(47144, 15)
Index(['temperature_f', 'resp_rate', 'mechanical_ventilation', 'oxygenthrapy',
       'hr', 'sbp', 'shorkindex', 'shorkindex_age', 'spo2_last', 'spo2_mean',
       'paco2', 'bun', 'cr_diff', 'gcs', 'die_in_hosp'],
      dtype='object')


In [5]:
# Check descriptive statistics
data.describe()

,temperature_f,resp_rate,mechanical_ventilation,oxygenthrapy,hr,sbp,shorkindex,shorkindex_age,spo2_last,spo2_mean,paco2,bun,cr_diff,gcs,die_in_hosp
count,47144.000000,47144.000000,47144.000000,47144.000000,47144.000000,47144.000000,47144.000000,47144.000000,47144.000000,47144.000000,47144.000000,47144.000000,47144.000000,47144.000000,47144.000000
mean,-1461.323349,-192.058232,0.181041,0.189950,-118.342447,-398.059689,-510.660356,-467.938797,-123.183565,-110.806640,-4682.264148,-72.276790,-5787.220592,-4872.931656,0.110131
std,3648.840233,1437.806358,0.385056,0.392266,1415.549403,2228.581385,2202.822092,2212.815810,1473.053702,1433.962586,5008.798635,976.887639,4937.105495,5005.117119,0.313056
min,-9999.000000,-9999.000000,0.000000,0.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,0.000000
25%,96.999998,15.825000,0.000000,0.000000,71.000000,102.000000,0.567376,31.816383,95.000000,96.041667,-9999.000000,12.000000,-9999.000000,-9999.000000,0.000000
50%,97.914286,18.000000,0.000000,0.000000,83.000000,116.000000,0.695238,42.666667,97.000000,97.488630,21.000000,18.000000,-9999.000000,4.000000,0.000000
75%,98.620000,20.750000,0.000000,0.000000,95.000000,132.000000,0.837424,54.375000,99.000000,98.657143,37.000000,28.000000,-0.000310,15.000000,0.000000
max,100.000000,48.000000,1.000000,1.000000,201.000000,267.000000,6.758621,581.241379,100.000000,100.000000,100.000000,242.000000,0.047451,15.000000,1.000000


In [6]:
data.head(20)

,temperature_f,resp_rate,mechanical_ventilation,oxygenthrapy,hr,sbp,shorkindex,shorkindex_age,spo2_last,spo2_mean,paco2,bun,cr_diff,gcs,die_in_hosp
0,97.549999,17.324324,0,0,75.0,102,0.735294,55.882353,98,97.589744,30,15,-9.999000e+03,15,0
1,98.320001,25.352941,0,0,97.0,119,0.815126,39.126050,98,97.500000,-9999,20,1.013370e-03,15,0
2,97.650000,12.407407,0,0,94.0,127,0.740157,48.850394,100,99.185185,-9999,17,-1.459472e-03,15,0
3,98.330000,14.404255,0,0,99.0,177,0.559322,22.932203,100,97.707317,27,33,-5.190890e-04,15,1
4,98.357144,18.000000,0,0,101.0,109,0.926606,46.330275,97,96.500000,-9999,16,-1.269512e-03,13,0
5,97.909092,15.238095,0,0,73.0,142,0.514085,37.014085,96,96.531250,25,37,7.716450e-04,15,1
6,97.649998,15.320000,0,0,119.0,108,1.101852,44.074074,100,97.700000,29,16,1.075573e-03,15,0
7,-9999.000000,20.687500,0,0,96.0,106,0.905660,43.471698,96,98.176471,32,8,1.524148e-03,15,0
8,97.766666,20.558824,0,0,85.0,98,0.867347,40.765306,97,99.057143,40,12,7.814220e-04,3,0
9,99.316667,19.958333,0,0,106.0,139,0.762590,38.892086,96,96.833333,29,12,-1.558001e-03,15,0


In [7]:
# Check data types and number of not-null values for each column
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47144 entries, 0 to 47143
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   temperature_f           47144 non-null  float64
 1   resp_rate               47144 non-null  float64
 2   mechanical_ventilation  47144 non-null  int64  
 3   oxygenthrapy            47144 non-null  int64  
 4   hr                      47144 non-null  float64
 5   sbp                     47144 non-null  int64  
 6   shorkindex              47144 non-null  float64
 7   shorkindex_age          47144 non-null  float64
 8   spo2_last               47144 non-null  int64  
 9   spo2_mean               47144 non-null  float64
 10  paco2                   47144 non-null  int64  
 11  bun                     47144 non-null  int64  
 12  cr_diff                 47144 non-null  float64
 13  gcs                     47144 non-null  int64  
 14  die_in_hosp             47144 non-null

In [8]:
data.head(20)

,temperature_f,resp_rate,mechanical_ventilation,oxygenthrapy,hr,sbp,shorkindex,shorkindex_age,spo2_last,spo2_mean,paco2,bun,cr_diff,gcs,die_in_hosp
0,97.549999,17.324324,0,0,75.0,102,0.735294,55.882353,98,97.589744,30,15,-9.999000e+03,15,0
1,98.320001,25.352941,0,0,97.0,119,0.815126,39.126050,98,97.500000,-9999,20,1.013370e-03,15,0
2,97.650000,12.407407,0,0,94.0,127,0.740157,48.850394,100,99.185185,-9999,17,-1.459472e-03,15,0
3,98.330000,14.404255,0,0,99.0,177,0.559322,22.932203,100,97.707317,27,33,-5.190890e-04,15,1
4,98.357144,18.000000,0,0,101.0,109,0.926606,46.330275,97,96.500000,-9999,16,-1.269512e-03,13,0
5,97.909092,15.238095,0,0,73.0,142,0.514085,37.014085,96,96.531250,25,37,7.716450e-04,15,1
6,97.649998,15.320000,0,0,119.0,108,1.101852,44.074074,100,97.700000,29,16,1.075573e-03,15,0
7,-9999.000000,20.687500,0,0,96.0,106,0.905660,43.471698,96,98.176471,32,8,1.524148e-03,15,0
8,97.766666,20.558824,0,0,85.0,98,0.867347,40.765306,97,99.057143,40,12,7.814220e-04,3,0
9,99.316667,19.958333,0,0,106.0,139,0.762590,38.892086,96,96.833333,29,12,-1.558001e-03,15,0


In [9]:
# Check class distribution
data['die_in_hosp'].value_counts()

0    41952
1     5192
Name: die_in_hosp, dtype: int64

# Data Split
- train set : test set = 7:3
- Stratified split using labels

In [10]:
X = data.drop('die_in_hosp', axis=1)  #열컬럼에서 drop 한다고 알려줌
y = data['die_in_hosp']

In [11]:
#test size 30%, stratify y의 비율을 유지한다는 이야기, 시드고정
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=True, stratify=y,random_state=0)

In [12]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33000 entries, 35874 to 17374
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   temperature_f           33000 non-null  float64
 1   resp_rate               33000 non-null  float64
 2   mechanical_ventilation  33000 non-null  int64  
 3   oxygenthrapy            33000 non-null  int64  
 4   hr                      33000 non-null  float64
 5   sbp                     33000 non-null  int64  
 6   shorkindex              33000 non-null  float64
 7   shorkindex_age          33000 non-null  float64
 8   spo2_last               33000 non-null  int64  
 9   spo2_mean               33000 non-null  float64
 10  paco2                   33000 non-null  int64  
 11  bun                     33000 non-null  int64  
 12  cr_diff                 33000 non-null  float64
 13  gcs                     33000 non-null  int64  
dtypes: float64(7), int64(7)
memory usa

# Make untuned XGBoost model

파이썬에서 두가지 인터페이스 존재

기존 인터페이스와 scilearn 과 호환된 인터페이스

오늘 실습은 scikit-learn API 에서의 하이퍼 파라미터로 확인할 예정


In [13]:
xgb = XGBClassifier()
xgb.fit(X_train, y_train)

XGBClassifier()

In [14]:
print(xgb)

XGBClassifier()


### 메모
학습이랑 테스트 둘다 평가할것을 권장함
- goodnees of fit의 관점
- 성능 관점

In [15]:
y_train_pred = xgb.predict(X_train)
y_train_prob = xgb.predict_proba(X_train)

y_test_pred  = xgb.predict(X_test)
y_test_prob = xgb.predict_proba(X_test)

In [16]:
y_train_prob

array([[0.9332347 , 0.06676532],
       [0.8294436 , 0.17055641],
       [0.9803089 , 0.01969112],
       ...,
       [0.918481  , 0.08151904],
       [0.95499104, 0.04500896],
       [0.96591413, 0.03408586]], dtype=float32)

## 학습 데이터의 성능 확인
- goodnees of fit의 관점

In [18]:
print('Accuracy: %.4f' % metrics.accuracy_score(y_train, y_train_pred))
print('AUC: %.4f' % metrics.roc_auc_score(y_train, y_train_prob[:, 1]))
print(metrics.classification_report(y_train, y_train_pred))

Accuracy: 0.9172
AUC: 0.8858
              precision    recall  f1-score   support

           0       0.92      0.99      0.96     29366
           1       0.80      0.33      0.47      3634

    accuracy                           0.92     33000
   macro avg       0.86      0.66      0.71     33000
weighted avg       0.91      0.92      0.90     33000



## test 데이터의 성능 평가

In [19]:
print('Accuracy: %.4f' % metrics.accuracy_score(y_test, y_test_pred))
print('AUC: %.4f' % metrics.roc_auc_score(y_test, y_test_prob[:, 1]))
print(metrics.classification_report(y_test, y_test_pred))

Accuracy: 0.9147
AUC: 0.8677
              precision    recall  f1-score   support

           0       0.92      0.99      0.95     12586
           1       0.77      0.32      0.45      1558

    accuracy                           0.91     14144
   macro avg       0.85      0.65      0.70     14144
weighted avg       0.91      0.91      0.90     14144



# Fine tune the XGBoost model
- Grid search

In [20]:
def fitting_cv(model, param, kfold, train_input, train_target, scoring='roc_auc', n_jobs=1, tracking=True):
    '''
    [Parameters]
    - model: A tuple like ('name', MODEL)
    - param
    - scoring: neg_mean_absolute_error, neg_mean_squared_error, neg_median_absolute_error, r2
               (http://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter)
    - n_jobs: default as -1 (if it is -1, all CPU cores are used to train and validate models)
    - tracking: whether trained model's name and duration time are printed
    '''
    
    name = model[0]
    estimator = model[1]
    if tracking:
        start_time = datetime.now()
        print("[%s] Start parameter search for model '%s'" % (start_time, name))
        gridsearch = GridSearchCV(estimator=estimator, param_grid=param, cv=cv_idx, scoring=scoring, n_jobs=n_jobs)
        gridsearch.fit(train_input, train_target)
        end_time = datetime.now()
        duration_time = (end_time - start_time).seconds
        print("[%s] Finish parameter search for model '%s' (time: %d seconds)" % (end_time, name, duration_time))
        print()
    else:
        gridsearch = GridSearchCV(estimator=estimator, param_grid=param, cv=kfold, scoring=scoring, n_jobs=n_jobs)
        gridsearch.fit(train_input, train_target)
    
    return gridsearch   


In [21]:
?XGBClassifier

In [22]:
param = {
    'learning_rate': [0.01, 0.03],
    'max_depth': [2, 3, 4], #bosting이라 weak 하게 주는거임
    'n_estimators': [100, 200, 300], #몇단계까지 갈거냐랑 동일함 
    'subsample': [0.8, 1], #bootstrap 데이터 샘프링 비율
    'colsample_bytree': [0.8, 1],  #bootstrap 컬럼 샘프링 비율
    'reg_lambda' : [0.001, 0.005]
}
# 144개의 후보군이라 많음

In [23]:
model = ('XGB', XGBClassifier(random_state=1234, tree_method='gpu_hist')) #gpu modeling gpu_hist 주면된다.
param = {
    'learning_rate': [0.01, 0.03]
}

In [24]:
model = XGBClassifier(random_state=0)
grid = GridSearchCV(estimator=model, param_grid=param,cv=5,n_jobs=-1)

In [25]:
grid.fit(X_train, y_train) #10번 학습함

GridSearchCV(cv=5, estimator=XGBClassifier(), n_jobs=-1,
             param_grid={'learning_rate': [0.01, 0.03]})

In [26]:
grid.best_params_

{'learning_rate': 0.03}

In [27]:
grid.best_score_

0.9060909090909091

In [28]:
grid.cv_results_ #CV 상세 이력

{'mean_fit_time': array([2.20319128, 2.09387574]),
 'std_fit_time': array([0.22833804, 0.0218477 ]),
 'mean_score_time': array([0.02207866, 0.02250333]),
 'std_score_time': array([0.0021056 , 0.00281973]),
 'param_learning_rate': masked_array(data=[0.01, 0.03],
              mask=[False, False],
        fill_value='?',
             dtype=object),
 'params': [{'learning_rate': 0.01}, {'learning_rate': 0.03}],
 'split0_test_score': array([0.90348485, 0.90712121]),
 'split1_test_score': array([0.9019697 , 0.90681818]),
 'split2_test_score': array([0.90469697, 0.90621212]),
 'split3_test_score': array([0.90121212, 0.90818182]),
 'split4_test_score': array([0.8980303 , 0.90212121]),
 'mean_test_score': array([0.90187879, 0.90609091]),
 'std_test_score': array([0.00227091, 0.00208497]),
 'rank_test_score': array([2, 1], dtype=int32)}

In [29]:
best_model = grid.best_estimator_
y_test_pred_best = best_model.predict(X_test)
y_test_prob_best = best_model.predict_proba(X_test)
print('Accuracy: %.4f' % metrics.accuracy_score(y_test, y_test_pred_best))
print('AUC: %.4f' % metrics.roc_auc_score(y_test, y_test_prob_best[:, 1]))
print(metrics.classification_report(y_test, y_test_pred_best))

Accuracy: 0.9070
AUC: 0.8441
              precision    recall  f1-score   support

           0       0.91      0.99      0.95     12586
           1       0.79      0.21      0.33      1558

    accuracy                           0.91     14144
   macro avg       0.85      0.60      0.64     14144
weighted avg       0.90      0.91      0.88     14144



In [30]:
best_model.feature_importances_

array([0.03438814, 0.1097669 , 0.06382191, 0.04431625, 0.05614719,
       0.04464659, 0.12498568, 0.09823883, 0.06820551, 0.04445333,
       0.04767288, 0.22430338, 0.03905343, 0.        ], dtype=float32)

In [31]:
import numpy as np
np.array(X_train.columns)

array(['temperature_f', 'resp_rate', 'mechanical_ventilation',
       'oxygenthrapy', 'hr', 'sbp', 'shorkindex', 'shorkindex_age',
       'spo2_last', 'spo2_mean', 'paco2', 'bun', 'cr_diff', 'gcs'],
      dtype=object)

In [32]:
pd.DataFrame(np.array(X_train.columns),np.array(best_model.feature_importances_))

,0
0.034388,temperature_f
0.109767,resp_rate
0.063822,mechanical_ventilation
0.044316,oxygenthrapy
0.056147,hr
0.044647,sbp
0.124986,shorkindex
0.098239,shorkindex_age
0.068206,spo2_last
0.044453,spo2_mean


In [35]:
!pip install xgboost
!pip install sklearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2344 sha256=b1b161622149c965af211a414f8ed9069468b6fcb6f23b41f3a354ebc6b6555f
  Stored in directory: /root/.cache/pip/wheels/14/25/f7/1cc0956978ae479e75140219088deb7a36f60459df242b1a72
Successfully built sklearn


In [39]:
# Generating cross-validation index
cv_idx = StratifiedKFold(n_splits=5, shuffle=True, random_state=1234)

In [40]:
result = fitting_cv(model=model, param=param, kfold=cv_idx, train_input=X_train, train_target=y_train, n_jobs=-1)

TypeError: ignored

In [ ]:
# http://aispiration.com/model/model-python-xgboost-hyper.html

In [ ]:
print(result.best_score_)

In [ ]:
print(result.best_params_)

In [ ]:
tuned_xgb = result.best_estimator_
print(tuned_xgb)

In [ ]:
y_train_pred = tuned_xgb.predict(X_train)
y_train_prob = tuned_xgb.predict_proba(X_train)

y_test_pred  = tuned_xgb.predict(X_test)
y_test_prob = tuned_xgb.predict_proba(X_test)

In [ ]:
print('Accuracy: %.4f' % metrics.accuracy_score(y_train, y_train_pred))
print('AUC: %.4f' % metrics.roc_auc_score(y_train, y_train_prob[:, 1]))
print(metrics.classification_report(y_train, y_train_pred))

In [ ]:
print('Accuracy: %.4f' % metrics.accuracy_score(y_test, y_test_pred))
print('AUC: %.4f' % metrics.roc_auc_score(y_test, y_test_prob[:, 1]))
print(metrics.classification_report(y_test, y_test_pred))

# Compare other models
- Logistic Regression
- Random Forest
- Feed-forward Network
- Support Vector Machine
- XGBoost

In [41]:
models = []
params = []

In [42]:
model = ('LR', LogisticRegression(solver='liblinear', multi_class='auto', random_state=2072))
param = {
    'C': [0.01, 0.1, 1.0, 5.0, 10.0, 100.0],
    'class_weight': [None, 'balanced']
}

models.append(model)
params.append(param)

In [43]:
model = ('RF', RandomForestClassifier(random_state=2072))
param = {
    'n_estimators': [20, 30, 40, 50, 60, 100],
    'max_features': [3, 4, 5, 6, 7],
    'class_weight': [None, 'balanced']
}

models.append(model)
params.append(param)

In [44]:
model = ('MLP', MLPClassifier(random_state=2072, max_iter=2000, batch_size=32))
param = {
    'hidden_layer_sizes': [(10, ), (15, ), (10, 10)],
    'alpha': [0.001, 0.05],
    'learning_rate_init': [0.005, 0.01]
}

models.append(model)
params.append(param)

In [45]:
model = ('SVC', SVC(random_state=2072))
param = [
    {'kernel': ['linear'], 'C': [0.01, 0.1, 1, 10, 100, 100]},
    {'kernel': ['rbf'], 'C': [0.01, 0.1, 1, 10, 100, 100], 'gamma': [0.01, 0.05, 0.1, 0.5, 1.0]},
    {'kernel': ['poly'], 'C': [0.01, 0.1, 1, 10, 100, 100], 'degree': [2, 3]}
]

models.append(model)
params.append(param)

In [46]:
model = ('XGB', XGBClassifier(random_state=2072))
param = {
    'learning_rate': [0.01, 0.03, 0.05],
    'max_depth': [2, 3, 4],
    'n_estimators': [100, 200, 300],
    'subsample': [0.8, 1],
    'colsample_bytree': [0.8, 1]
}

models.append(model)
params.append(param)

In [47]:
# Generating cross-validation index
cv_idx = StratifiedKFold(n_splits=10, shuffle=True, random_state=2072)

In [48]:
results = []
for model, param in zip(models, params):
    result = fitting_cv(model=model, param=param, kfold=cv_idx, train_input=X_train, train_target=y_train, n_jobs=-1)
    results.append(result)

[2022-12-20 09:42:51.516407] Start parameter search for model 'LR'
[2022-12-20 09:44:15.814042] Finish parameter search for model 'LR' (time: 84 seconds)

[2022-12-20 09:44:15.814239] Start parameter search for model 'RF'


KeyboardInterrupt: ignored